In [1]:
from config import config
import nest_asyncio
import ssl
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

# Enable nested event loops (required for Jupyter)
nest_asyncio.apply()

# Setup GraphQL transport and client
transport = AIOHTTPTransport(url=config.THEGRAPH_ENDPOINT, ssl=ssl.create_default_context())
client = Client(transport=transport, fetch_schema_from_transport=False)

# GraphQL query to get top pools
QUERY_TOP_POOLS = gql("""
  query topPools($n:Int!) {
    pools(first: $n,
          orderBy: totalValueLockedUSD,
          orderDirection: desc) {
      id
      feeTier
      token0 { id symbol name decimals }
      token1 { id symbol name decimals }
      totalValueLockedUSD
      volumeUSD
    }
  }
""")

# Query poolDayData for latest volumeUSD
QUERY_DAYDATA = gql("""
  query poolDayData($id: String!) {
    poolDayDatas(
      first: 1,
      orderBy: date,
      orderDirection: desc,
      where: { pool: $id }
    ) {
      date
      volumeUSD
    }
  }
""")

# Async logic to fetch pools and their daily volume
async def fetch_filtered_pools(min_volume=1000, top_n=100):

    async with client as session:
        # We will fetch top 1000 pools by totalValueLockedUSD and then filter top_n pools by daily volume
        raw = await session.execute(QUERY_TOP_POOLS, variable_values={"n": 1000})
        pools = raw["pools"]

        result = []
        for p in pools:
            pool_id = p["id"]

            # Query last day volume
            day_data = await session.execute(QUERY_DAYDATA, variable_values={"id": pool_id})
            items = day_data["poolDayDatas"]
            if not items:
                continue
            
            # Filter pools with daily volume greater than min_volume
            volume = float(items[0]["volumeUSD"])
            if volume < min_volume:
                continue

            # Add pool data to result
            result.append(p)

            if len(result) == top_n:
                break

        return result
    

pools_raw = await fetch_filtered_pools(min_volume=1000, top_n=100)

print(f"Fetched {len(pools_raw)} pools with daily volume > 1000 USD")

print("Closing GraphQL client session")
try:
    await client.close_async()
    print("GraphQL client session closed successfully.")
except Exception as e:
    print(f"Error closing GraphQL client session: {e}")


Fetched 100 pools with daily volume > 1000 USD
Closing GraphQL client session
GraphQL client session closed successfully.


In [2]:
import pandas as pd

def convert_to_dataframe(pools_raw):
    """
    Converts the raw pool data to a pandas DataFrame.
    Args:
        pools_raw (list): List of raw pool data.
    Returns:
        pd.DataFrame: DataFrame containing the pool data.
    """
    pools = []
    for p in pools_raw:
        pool = {
            "id": str(p["id"]).lower(),
            "feeTier": int(p["feeTier"]),
            "token0_id": str(p["token0"]["id"]).lower(),
            "token0_symbol": str(p["token0"]["symbol"]).upper(),
            "token0_name": str(p["token0"]["name"]).upper(),
            "token1_id": str(p["token1"]["id"]).lower(),
            "token1_symbol": str(p["token1"]["symbol"]).upper(),
            "token1_name": str(p["token1"]["name"]).upper(),
            "volumeUSD": float(p["volumeUSD"]),
            "totalValueLockedUSD": float(p["totalValueLockedUSD"])
        }
        pools.append(pool)

    return pd.DataFrame(pools)

pools_df = convert_to_dataframe(pools_raw)

pools_df.head()

,id,feeTier,token0_id,token0_symbol,token0_name,token1_id,token1_symbol,token1_name,volumeUSD,totalValueLockedUSD
0,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,500,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD COIN,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,WRAPPED ETHER,5.584420e+11,4.051979e+08
1,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,3000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD COIN,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,WRAPPED ETHER,8.435547e+10,3.206366e+08
2,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,3000,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,WRAPPED BTC,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,WRAPPED ETHER,3.208979e+10,2.798855e+08
3,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,3000,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,WRAPPED BTC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,USD COIN,2.412572e+10,2.106167e+08
4,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,WRAPPED ETHER,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,TETHER USD,4.715590e+10,1.992544e+08


In [3]:
# find all wrapped tokens
wrapped_tokens = set()
for _, row in pools_df.iterrows():
    if "WRAPPED" in row["token0_name"]:
        wrapped_tokens.add(row["token0_symbol"])
    if "WRAPPED" in row["token1_name"]:
        wrapped_tokens.add(row["token1_symbol"])

print(f"Found {len(wrapped_tokens)} wrapped tokens:")
print(wrapped_tokens)

Found 11 wrapped tokens:
{'SOL', 'WSTETH', 'WXTM', 'WTAO', 'WM', 'WETH', 'CBBTC', 'WDOGE', 'WBTC', 'TONCOIN', 'WEETH'}


In [4]:
import ccxt

# wrapped tokens that can be converted to native tokens 1:1
WRAPPED_TOKEN_MAP = {
    "WETH": "ETH",
    "WBTC": "BTC",
    "WDOGE": "DOGE",
    "WBNB": "BNB",
    "WMATIC": "MATIC",
    "WTAO": "TAO",
    "WSTETH": "ETH",
    "WEETH": "ETH",
    "CBBTC": "BTC",
    "TONCOIN": "TON"
}

#TODO: maybe check other wrapped tokens and add more

def unwrap_token(symbol):
    return WRAPPED_TOKEN_MAP.get(symbol, symbol)


# download pairs from binance
exchange = ccxt.binance()
markets = exchange.load_markets()
binance_symbols = {
    symbol for symbol, market in markets.items()
    if market.get("spot") and market.get("active")
}

# find all pairs on binance
results = []

for _, row in pools_df.iterrows():
    t0_symbol = row["token0_symbol"]
    t1_symbol = row["token1_symbol"]

    # unwrap tokens
    u0 = unwrap_token(t0_symbol)
    u1 = unwrap_token(t1_symbol)

    # check if the pair is on binance
    pair1 = f"{u0}/{u1}"
    pair2 = f"{u1}/{u0}"

    binance_pair, reverse_price = None, None
    if pair1 in binance_symbols:
        binance_pair = pair1
        reverse_price = 0
    elif pair2 in binance_symbols:
        binance_pair = pair2
        reverse_price = 1

    if binance_pair:
        results.append({
            "uniswap_pair": f"{t0_symbol}/{t1_symbol}",
            "uniswap_pool_id": row["id"],
            "uniswap_fee_tier": row["feeTier"],
            "binance_pair": binance_pair,
            "reverse_price": reverse_price
        })

# Create DataFrame from results
arbitrage_pairs_df = pd.DataFrame(results)

print(f"Found {len(arbitrage_pairs_df)} arbitrage pairs on Binance.")

Found 25 arbitrage pairs on Binance.


In [5]:
# Save to CSV
arbitrage_pairs_df.to_csv("arbitrage_pairs.csv", index=False)
print("Saved arbitrage pairs to arbitrage_pairs.csv")

Saved arbitrage pairs to arbitrage_pairs.csv
